In [3]:
from langchain.schema.runnable import RunnableLambda

def word_counter(text):
    return len(text.split(" "))


runnable_word_counter = RunnableLambda(word_counter)

result = runnable_word_counter.invoke("hey what is your name")

print(result)

5


In [6]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate
from langchain.schema.runnable import RunnableSequence, RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os

load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1",
    task = "text-generation",
    huggingfacehub_api_token=os.getenv("HUGINGFACEHUB_API_TOKEN")
)

model = ChatHuggingFace(llm = llm)

parser = StrOutputParser()

prompt = PromptTemplate(
    template= "write a joke about the {topic}",
    input_keys=["topic"]
)

joke_gen_seq_chain = RunnableSequence(prompt, model, parser)

# parallel_chain = RunnableParallel({
#     "joke" : RunnablePassthrough(),
#     "word_count" : RunnableLambda(word_counter)
# })

parallel_chain = RunnableParallel({
    "joke" : RunnablePassthrough(),
    "word_count" : RunnableLambda(lambda x : len(x.split(" ")))
})

final_chain = RunnableSequence(joke_gen_seq_chain, parallel_chain)

result = final_chain.invoke({"topic" : "cricket"})

final_result = f"""
Joke: {result["joke"]}
Word Count: {result["word_count"]}
"""

print(final_result)



Joke:  Why don't crickets watch movies? Because they prefer to stick to their own "series" - a cricket "series" is when a group of crickets take turns telling jokes and stories all night long! Just imagine the amount of "cricket critiques" they have after each performance! Sorry for the bad puns, I just couldn't resist!
Word Count: 55



In [8]:
final_chain.get_graph().print_ascii()

           +-------------+            
           | PromptInput |            
           +-------------+            
                   *                  
                   *                  
                   *                  
          +----------------+          
          | PromptTemplate |          
          +----------------+          
                   *                  
                   *                  
                   *                  
         +-----------------+          
         | ChatHuggingFace |          
         +-----------------+          
                   *                  
                   *                  
                   *                  
         +-----------------+          
         | StrOutputParser |          
         +-----------------+          
                   *                  
                   *                  
                   *                  
  +--------------------------------+  
  | Parallel<joke,word_co